In [23]:
from sqlalchemy import create_engine
from sqlalchemy import inspect
import pandas as pd

In [24]:
engine = create_engine("sqlite:///mta_data.db")

In [25]:
insp = inspect(engine)
print(insp.get_table_names())

['mta_data']


In [26]:
pd.read_sql('SELECT * FROM mta_data;', engine)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/28/2021,00:00:00,REGULAR,7625998,2609113
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/28/2021,04:00:00,REGULAR,7626014,2609117
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/28/2021,08:00:00,REGULAR,7626024,2609135
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/28/2021,12:00:00,REGULAR,7626086,2609191
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/28/2021,16:00:00,REGULAR,7626206,2609221
...,...,...,...,...,...,...,...,...,...,...,...
2932340,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/04/2021,05:00:00,REGULAR,5554,577
2932341,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/04/2021,09:00:00,REGULAR,5554,577
2932342,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/04/2021,13:00:00,REGULAR,5554,577
2932343,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/04/2021,17:00:00,REGULAR,5554,577


In [29]:
turnstile_daily = pd.read_sql("SELECT [C/A] || ' ' || UNIT || ' ' || SCP AS UniqueID, STATION, DATE, Abs(max(ENTRIES)-min(ENTRIES) ) AS DailyEntries, Abs(max(EXITS)-min(EXITS) )AS DailyExits FROM mta_data GROUP BY UniqueID, STATION, DATE;", engine)
#calculates net number of entries and exits by turnstile per day

In [33]:
turnstile_daily.head()

,UniqueID,STATION,DATE,DailyEntries,DailyExits
0,A002 R051 02-00-00,59 ST,05/29/2021,287,114
1,A002 R051 02-00-00,59 ST,05/30/2021,197,97
2,A002 R051 02-00-00,59 ST,05/31/2021,281,81
3,A002 R051 02-00-00,59 ST,06/01/2021,525,266
4,A002 R051 02-00-00,59 ST,06/02/2021,504,274


In [36]:
turnstile_daily_total = pd.read_sql("SELECT UniqueID, STATION, DATE, DailyEntries+DailyExits AS DailyTraffic FROM (SELECT [C/A] || ' ' || UNIT || ' ' || SCP AS UniqueID, STATION, DATE, Abs(max(ENTRIES)-min(ENTRIES)) AS DailyEntries, Abs(max(EXITS)-min(EXITS))AS DailyExits FROM mta_data GROUP BY UniqueID, STATION, DATE) WHERE DailyEntries <4000 AND DailyExits <4000;", engine)
#calculates total traffic (entries and exits by turnstile per day)

In [37]:
turnstile_daily_total.head()

,UniqueID,STATION,DATE,DailyTraffic
0,A002 R051 02-00-00,59 ST,05/29/2021,401
1,A002 R051 02-00-00,59 ST,05/30/2021,294
2,A002 R051 02-00-00,59 ST,05/31/2021,362
3,A002 R051 02-00-00,59 ST,06/01/2021,791
4,A002 R051 02-00-00,59 ST,06/02/2021,778


In [27]:
station_totals = pd.read_sql("SELECT STATION, SUM(DailyTraffic) AS TotalDailyTraffic FROM(SELECT UniqueID, STATION, DATE, DailyEntries+DailyExits AS DailyTraffic FROM (SELECT [C/A] || ' ' || UNIT || ' ' || SCP AS UniqueID, STATION, DATE, Abs(max(ENTRIES)-min(ENTRIES) ) AS DailyEntries, Abs(max(EXITS)-min(EXITS) )AS DailyExits FROM mta_data GROUP BY UniqueID, STATION, DATE) WHERE DailyEntries <4000 AND DailyExits <4000) GROUP BY STATION ORDER BY TotalDailyTraffic DESC;", engine)
#Gives total station traffic during observed period 

In [38]:
station_totals.head(10)

,STATION,TotalDailyTraffic
0,34 ST-PENN STA,8978720
1,34 ST-HERALD SQ,6793485
2,GRD CNTRL-42 ST,6692149
3,86 ST,5919069
4,14 ST-UNION SQ,5818136
5,23 ST,5398562
6,42 ST-PORT AUTH,5338444
7,125 ST,5234886
8,FULTON ST,4795110
9,TIMES SQ-42 ST,4767529
